In [0]:
import json
import mysql.connector

In [0]:
# Class for DB connection
# nothing here
class DB:
  def __init__(self, user='root', password='root', host='db', db='metrics', port='3306'):
    self.user = user
    self.password = password
    self.host = host
    self.db = db
    self.port = port
  
  def __enter__(self):
    # Connect to DB and create a cursor
    try:
      self.conn = mysql.connector.connect(user=self.user, password=self.password,
                                   host=self.host, database=self.db,
                                   port=self.port)
    
      self.cur = self.conn.cursor(buffered=True)
      return self.cur
    except Exception as e:
      return f"Failed to connect to DB: {e}"

  def __exit__(self, type, value, traceback):
    self.cur.close()
    self.conn.close()

# Errors
class NotFound(Exception):
  def __init__(self, type_name, id):
    self.msg = f'There are no item in {type_name} with the id {id}.'

class WrongTable(Exception):
  def __init__(self, table):
    self.msg = f'Check Table name. Couldn\'t find {table}.'


# Helper functions
def to_json(cur, data):
  json_data = []
  row_headers = [x[0] for x in cur.description]
  for result in data:
    json_data.append(dict(zip(row_headers, result)))
  
  try:
    return json.dumps(json_data)
  except Exception:
    return json_data


def get_obj(table, id):
  table = table
  db_conn = DB()
  with db_conn as db:
    field = 'doi'
    if table == 'bibliometrics':
      field = 'issn1'
    q = f"SELECT * FROM {table} WHERE {field} = '{id}';"
    
    try:
      db.execute(q)
      result = db.fetchall()
    except Exception as e:
      if e.errno == 1146:
        raise WrongTable(table)

    if len(result) < 1:
      raise NotFound(table, id)

    return to_json(db, result)


def print_response(response, content_type = 'application/json'):
  if content_type == 'application/json':
    return {
        'success': True,
        'data': response
    }
  else:
    return ':('

def create_error(code, error_type, message):
  return {
      'success': False,
      'code': code,
      'type': error_type,
      'message': message
  }

# GET /:table/record/:id
*Get JSON record for given item in table*
* table (required): the name of the table to query
* id (required): the ID of the object. Either doi or ISSN, depending on the table

In [0]:
REQUEST = json.dumps({
    'path': {
        'table': '1',
        'id': '1'
    }
})

In [0]:
# GET /:table/record/:id
try:
  req = json.loads(REQUEST)
  obj_id = req['path']['id']
  table = req['path']['table']
  response = print_response(get_obj(table, obj_id))
except NotFound as e:
  response = create_error(405, 'NotFound', e.msg)
except WrongTable as e:
  response = create_error(405, 'TableNameError', e.msg)
except Exception as e:
  response = e.errno
finally:
  print(response)